In [ ]:
import plotly.express as px
from glob import glob
import os
import pandas as pd
import csv
import numpy as np
import seaborn as sns
from collections import defaultdict
import plotly.graph_objects as go
from matplotlib import pyplot as plt

In [ ]:
img_dir = 'images'
os.makedirs(img_dir, exist_ok=True)

# DFIV Results

In [ ]:
DUMP_DIR = '../dumps'
dir = 'dfiv_low_dim_parcs'

In [ ]:
def get_results(dir):
    dfs = []
    root = os.path.join(DUMP_DIR, dir)
    for subdir in os.listdir(root):
        if not os.path.isdir(os.path.join(root, subdir)):
            continue
        props =  [item.split(':') for item in subdir.split('-')]
        props = {item[0]: [item[1]] for item in props}

        with open (os.path.join(root, subdir, 'result.csv'), 'r') as f:
            reader = csv.reader(f)
            mse = [float(row[0]) for row in list(reader)]
            
        n = len(mse)
        props = {key: val * n for key, val in props.items()}
        props.update({'mse': mse, 'case': [dir] * n})
        dfs.append(pd.DataFrame(props))
    return pd.concat(dfs)

In [ ]:
df_parcs =  get_results('dfiv_low_dim_parcs')
df_parcs = df_parcs.fillna(0)
# set(df.sigma)
df_parcs.sigma = ['0' if val == 'None' else val for val in df_parcs.sigma]
set(df_parcs.sigma)

In [ ]:
df_orignal = get_results('dfiv_low_dim')
df_orignal

In [ ]:
df_comb = pd.concat([df_parcs, df_orignal])
df_comb
df_comb = df_comb.fillna('0')
df_comb = df_comb[df_comb['rho'] == '0.5']

In [ ]:
df_comb.loc[df_comb['case'] == 'dfiv_low_dim', 'case'] = 'Original'
df_comb.loc[df_comb['case'] != 'Original', 'case'] = "PARCS'"

In [ ]:
df_avg = df_comb.groupby(['data_size', 'rho', 'case', 'sigma']).mean()
df_avg = df_avg.reset_index()
df_avg = df_avg.fillna(0)
df_avg = df_avg[df_avg['rho'] == '0.5']
df_avg

In [ ]:
df_std = df_comb.groupby(['data_size', 'rho', 'case', 'sigma'])['mse'].std()
df_std = df_std.reset_index()
df_std[df_std['rho'] == '0.5']

In [ ]:
df_avg['std'] = df_std['mse']
df_avg

In [ ]:
# df_comb = df_comb.loc[df_comb['data_size'] != '1000']

In [ ]:
fig = px.box(df_comb, x='sigma', y='mse', color = 'case', points='all',
                facet_col='data_size', facet_col_wrap=3,
                log_y=True, range_y=[90,3*10**3],
                height = 400, width = 1200,
                category_orders = {'sigma': ['0', '0.2', '1', '1.8'],
                                   'data_size': ['1000', '5000', '10000'],
                                   'rho': ['0.1', '0.5', '0.9']},
                labels={
                     "data_size": "N",
                     "mse": "MSE",
                     "case": "Case",
                     'sigma': 'Sigma'
                 })
fig.update_layout(
    font=dict(
        size=30
    ))
fig.update_yaxes(dtick=1)
fig.update_traces(width=0.6)
fig.show()
fig.write_image(f"{img_dir}/mean squared error box.pdf")

In [ ]:
fig = px.bar(df_avg, x='sigma', y='mse', barmode='group', color = 'case',
                facet_col='data_size', facet_col_wrap=3,
                height = 400, width = 1200,
                category_orders = {'sigma': ['0', '0.2', '1', '1.8'],
                                   'data_size': ['5000', '10000'],
                                   'data_size': ['1000', '5000', '10000'],
                                   'rho': ['0.1', '0.5', '0.9']},
                error_y = 'std',
                labels={
                     "data_size": "N",
                     "mse": "MSE",
                     "case": "Case",
                     'sigma': 'Sigma'
                 })
fig.update_xaxes(type='category')
fig.update_layout(
    font=dict(
        size=25
    )
)
fig.update_traces(width=0.3)
fig.show()
fig.write_image(f"{img_dir}/mean squared error bar.pdf")

In [ ]:
# fig = sns.FacetGrid(df, col = 'rho')
# fig.map_dataframe(sns.boxplot, x='data_size', y='mse', hue='case')
# plt.yscale('log')
# fig.set(yscale="log")
# fig.tight_layout()
# fig.add_legend()

# DFIV vs SparseIV

In [ ]:
case = 'case_vanilla'

In [ ]:
cases = [case]
res_dir = '../../spaceIV/results/comp/'
dfs = {case: pd.read_csv(os.path.join(res_dir, f'{case}.csv')) for case in cases}
df_sparseiv = dfs[case]
df_sparseiv['model'] = 'spaceIV'

In [ ]:
oi = {case: case}
df_dfiv =  get_results(oi)
df_dfiv['model'] = 'DFIV'

In [ ]:
df_merge = pd.concat([df_sparseiv,df_dfiv], axis=0)
df_merge

In [ ]:
fig = px.box(df_merge, x='model', y='mse',
             points="all",
             height = 400, width = 800,
             labels={
                     "mode": "Algorithm",
                     "mse": "Mean Squared Error",
                 })
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()
fig.write_image(f"{img_dir}/spaceiv_vanilla.pdf")

# Viz Demand Dataset

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.rcParams.update({
    "figure.figsize": [7.50, 3.50],
    "figure.autolayout": True,
    "text.usetex": True,
    "font.family": "sans-serif",
    'font.size': 16
})

data_size = 1000

In [ ]:
def psi(t: np.ndarray) -> np.ndarray:
    return 2 * ((t - 5) ** 4 / 600 + np.exp(-4 * (t - 5) ** 2) + t / 10 - 2)


def f(p: np.ndarray, t: np.ndarray, s: np.ndarray) -> np.ndarray:
    return 100 + (10 + p) * s * psi(t) - 2 * p

t = np.linspace(0, 10, data_size)

plt.plot(t, psi(t), color='blue')
plt.title('Function $g(t) = 2((t-5)^4)/600 + \exp(-4(t-5)^2) + t/10 - 2)$')
plt.savefig(f'{img_dir}/time_function.pdf')
plt.show()

In [ ]:
min_t = 2.53379
min_psi_t = psi(min_t)
print(f'min psi(t): {min_psi_t}')
z = 3
min_p = 25 + min_psi_t * (z + 3) -3
print(f'min p: {min_p}')

max_t = 10
max_psi_t = psi(max_t)
print(f'max psi(t): {max_psi_t}')
z = 3
max_p = 25 + max_psi_t * (z + 3) + 3
print(f'max p: {max_p}')

In [ ]:
rng = np.random.default_rng(seed=0)
rho = 0.5
emotion = rng.choice(list(range(1, 8)), data_size)
time = rng.uniform(0, 10, data_size)
cost = rng.normal(0, 1.0, data_size)
noise_price = rng.normal(0, 1.0, data_size)
noise_demand = rho * noise_price + rng.normal(0, np.sqrt(1 - rho ** 2), data_size)
price = 25 + (cost + 3) * psi(time) + noise_price
structural: np.ndarray = f(price, time, emotion).astype(float)
outcome: np.ndarray = (structural + noise_demand).astype(float)

price_scale = (price - min_p) / (max_p - min_p)
sigmas = [0.2, 1, 1.8]
hetero_noise_demand = [rho * noise_price + rng.normal(0, price_scale * sigma, data_size) for sigma in sigmas]

In [ ]:
# fig = px.scatter(x=price, y=noise_demand, width=800, height=400)
# fig.update_traces(marker_size=10)
# fig.show()

In [ ]:
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# fig = make_subplots(rows=2, cols=2, subplot_titles=(r'$\text{Demand Variance} = 1-\rho^2$ = 0.75$',
#                                                     r'$\text{Demand Variance} = r(p) * 0.2$', 
#                                                     r'$\text{Demand Variance} = r(p)$', 
#                                                     r'$\text{Demand Variance} = $r(p) * 1.8$'))

# fig.add_trace(go.Scatter(x=price, y=noise_demand, mode="markers", marker=dict(color='blue', size=10)), row=1, col=1)

# fig.add_trace(go.Scatter(x=price, y=hetero_noise_demand[0], mode="markers", marker=dict(color='blue', size=10)), row=1, col=2)

# fig.add_trace(go.Scatter(x=price, y=hetero_noise_demand[1], mode="markers", marker=dict(color='blue', size=10)), row=2, col=1)

# fig.add_trace(go.Scatter(x=price, y=hetero_noise_demand[2], mode="markers", marker=dict(color='blue', size=10)), row=2, col=2)

# fig.update_layout(height=800, width=1200, title_text="", showlegend=False)

# fig.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))

axs[0, 0].scatter(price, noise_demand)
axs[0, 0].set_title(r'Demand Variance = $1-\rho ^2 = 0.75$')
axs[0, 0].set_ylabel('The Noise of Demand')
axs[0, 0].set_xlabel('Price')

axs[0, 1].scatter(price, hetero_noise_demand[0])
axs[0, 1].set_title(r'Demand Variance = $r(p) * 0.2$')
axs[0, 1].set_ylabel('The Noise of Demand')
axs[0, 1].set_xlabel('Price')

axs[1, 0].scatter(price, hetero_noise_demand[1])
axs[1, 0].set_title(r'Demand Variance = $r(p)$')
axs[1, 0].set_ylabel('The Noise of Demand')
axs[1, 0].set_xlabel('Price')

axs[1, 1].scatter(price, hetero_noise_demand[2])
axs[1, 1].set_title(r'Demand Variance = $r(p) * 1.8$')
axs[1, 1].set_ylabel('The Noise of Demand')
axs[1, 1].set_xlabel('Price')

fig.savefig(f'{img_dir}/demand_noise.pdf')